# README

`recsys_test.py` を jupyter notebook に雑に変換したもの．なので全然整ってない．

MovieLens の嗜好データを
[公式サイト](https://grouplens.org/datasets/movielens/100k/) 
からダウンロードした．README に詳しい説明あり．ファイルとかカラムとか．
[このサイト](https://yag.xyz/blog/2015/10/03/movielens-datasets/)
にもデータの簡単な紹介がある．いろいろなサイズのやつあって，とりあえず一番小さい(古い)やつにした．

これを使って協調フィルタリングを(なるべくスクラッチで)実装してみる．



# import and check

In [1]:
import numpy as np
import pandas as pd

dat = pd.read_table("/Users/rui/data/ml-100k/u.data", header=None)
dat.columns = ["user_id", "movie_id", "rating", "timestamp"]
dat
# dat.dtypes
# dat.isnull().sum()

# dat["user_id"].nunique()
# dat["movie_id"].nunique()
# dat["rating"].nunique()
# dat["rating"].value_counts()



,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [2]:
# movie = pd.read_table("/Users/rui/data/ml-100k/u.item", header=None, encoding="shift_jis")
# movie = pd.read_table("/Users/rui/data/ml-100k/u.item_utf8.txt", header=None)
movie = pd.read_csv('/Users/rui/data/ml-100k/u.item_utf8.txt', sep='|', header=None)
movie_mst = movie[[0,1]]
movie_mst.columns = ["movie_id", "movie_name"]
# エンコーディングで苦労した．参考：https://stackoverflow.com/questions/30752973/encoding-issues-while-reading-importing-csv-file-in-python3-pandas
# コピーしてVSCodeで開いたらutf-8にできて，読み込めた．
# カラム内容についてはREADMEを参照．

# movie = pd.read_table("/Users/rui/data/ml-100k/u.item", header=None, encoding="shift_jis")
# movie = pd.read_table("/Users/rui/data/ml-100k/u.item_utf8.txt", header=None)
movie = pd.read_csv('/Users/rui/data/ml-100k/u.item_utf8.txt', sep='|', header=None)
movie_mst = movie[[0,1]]
movie_mst.columns = ["movie_id", "movie_name"]
movie_mst
# エンコーディングで苦労した．参考：https://stackoverflow.com/questions/30752973/encoding-issues-while-reading-importing-csv-file-in-python3-pandas
# コピーしてVSCodeで開いたらutf-8にできて，読み込めた．
# カラム内容についてはREADMEを参照．


,movie_id,movie_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


# make rating matrix

In [3]:
ratemat = pd.DataFrame(
    index=dat["user_id"].unique(),
    columns=dat["movie_id"].unique()
)

for i in range(len(dat)):
    user = dat.iloc[i, 0]
    movie = dat.iloc[i, 1]
    rate = dat.iloc[i, 2]
    ratemat.at[user,movie] = rate
    

In [5]:
ratemat  # 評価値行列．
# ratemat.isnull().mean()  # どの映画も9割近くが欠損

242     0.875928
302     0.685048
377     0.986214
51      0.914104
346     0.866384
          ...   
1674    0.998940
1640    0.998940
1637    0.998940
1630    0.998940
1641    0.998940
Length: 1682, dtype: float64

In [9]:
ranking = ratemat.notnull().mean().reset_index()
# mean(), sum(), notnull().mean()
# でいろんな観点からランキング見れる．
ranking.columns = ["movie_id", "value"]
ranking = ranking.merge(movie_mst, on="movie_id").sort_values("value", ascending=False)
ranking.iloc[:10,:]

,movie_id,value,movie_name
357,50,0.618240,Star Wars (1977)
157,258,0.539767,Contact (1997)
49,100,0.538706,Fargo (1996)
52,181,0.537646,Return of the Jedi (1983)
95,294,0.514316,Liar Liar (1997)
289,286,0.510074,"English Patient, The (1996)"
60,288,0.506893,Scream (1996)
24,1,0.479321,Toy Story (1995)
652,300,0.457052,Air Force One (1997)
403,121,0.454931,Independence Day (ID4) (1996)


# define class for Collaborative Filtering

In [10]:
class CollaborativeFiltering:
    """ class for Collaborative Filtering
    とりあえずユーザー間型メモリベース協調フィルタリングのみ実装．
    他のタイプはまた今度．
    """

    def __init__(self, sim_metric="pearson", nearest_k=10):
        """initialize
        Args:
          sim_metric: 類似度の指標．とりあえずピアソン相関係数のみ実装．
          nearest_k: k-NN の k．
        """
        self.sim_metric = sim_metric
        self.nearest_k = nearest_k

    def _calc_similarity(self, user1, user2, R, sim_metric):
        """類似度の計算
        メソッド fit() から呼び出して使う．
        """
        tf1 = R.loc[user1,:].notnull()
        tf2 = R.loc[user2,:].notnull()
        common_item = (tf1 & tf2)[(tf1 & tf2)].index
        sim = R.loc[user1, common_item].astype(int).corr(
            R.loc[user2, common_item].astype(int),
            method=sim_metric
        )
        return sim

    def fit(self, R):
        """ユーザー類似度行列を計算
        Args:
          R: 評価値行列．行がユーザー，列がアイテム．
        モデルベース協調フィルタとの関連を考えると，類似度計算がモデルフィットに対応．
        """
        self.sim_mat = pd.DataFrame(index=R.index, columns=R.index)
        for u1 in R.index:
            for u2 in R.index:
                sim = self._calc_similarity(
                    u1, u2, R, self.sim_metric
                )
                if np.isnan(sim):
                    sim = 0
                self.sim_mat.at[u1, u2] = sim
        self.R = R

    def predict(self, user, item):
        """評価値を予測
        Args:
          user: アクティブユーザー(レコメンド対象)
          item: 評価値を予測したいアイテム
        神嶌先生のpdfの(9.2)式を実装．
        user の k-NN に item を評価した人が一人もいない場合，
        user が付けた評価値の平均値で予測される．まあ直感的といえば直感的．
        """
        sim_all = self.sim_mat.loc[user, :].drop(index=user)
        sim_neighbor = sim_all[sim_all.abs().sort_values(ascending=False).iloc[:self.nearest_k].index]
        clb = 0
        for idx in sim_neighbor.index:
            avg = self.R.loc[idx, :].mean()
            rsd = self.R.loc[idx, item] - avg
            if not np.isnan(rsd):
                clb = clb + rsd * sim_neighbor[idx]
        clb = clb / sim_neighbor.abs().sum()
        return self.R.loc[user].mean() + clb

    def recommend(self, user, top_n, item_mst, key):
        """レコメンドするアイテムを決定
        Args:
          user: アクティブユーザー(レコメンド対象)
          top_n: 上位何個までのアイテムを返すか．
          item_mst: アイテムマスタ
          key: マスタとの結合キー(つまりアイテムIDのカラム名)
        """
        item_list = self.R.loc[user, self.R.loc[user, :].isnull()].index
        score = pd.Series(index=item_list)
        for item in item_list:
            score.at[item] = self.predict(user=user, item=item)
        out = score.sort_values(ascending=False).iloc[:10].reset_index()
        out.columns = [key, "predicted_score"]
        out = out.merge(item_mst, on=key, how="left")
        return out



# perform collaborative filtering

In [15]:
colab_filt = CollaborativeFiltering(sim_metric="pearson", nearest_k=10)
# ハイパラ的な部分の設定．

ratemat_short = ratemat.iloc[:100, :]
# 計算時間きついので100人だけを対象にする．

colab_filt.fit(R=ratemat_short)
# モデルのフィット(=類似度の計算)

colab_filt.sim_mat
# colab_filt.sim_mat.at[196, 22]

# ratemat_short
# ratemat_short.isnull().mean().sort_values()

,196,186,22,244,166,298,115,253,305,6,...,94,128,44,264,41,82,262,174,43,84
196,1,0,0.358045,0.273091,0,0.190941,0.207514,-0.845154,0.266574,0.107721,...,0.356232,-0.172212,-0.607957,-0.333849,0.315063,0.396059,0.207853,-0.0448963,0.065407,0
186,0,1,0.0586643,-0.172359,0.757572,-0.13159,-0.158552,0.208848,-0.27769,0.107662,...,-0.0327805,0.246551,-0.119545,-0.390625,0,0.148713,-0.0953772,0.0801638,-0.128217,-0.287476
22,0.358045,0.0586643,1,0.183423,0.662266,0.271121,0.716801,0.134314,0.202106,0.572611,...,0.247889,-0.0194837,0.348299,0.698328,0.0661107,0.379565,0.250307,0.563688,0.0767775,0.601699
244,0.273091,-0.172359,0.183423,1,1,0.0489473,0.188793,0.0237823,0.170749,0.298476,...,0.387261,0.136644,0.176494,0.132324,0.319718,0.23055,0.215754,0.149548,0.229082,0.256278
166,0,0.757572,0.662266,1,1,-1,0,0.534522,-0.380304,0,...,0.111803,-0.197642,0,0.755929,-0.693375,-0.749269,0.866025,-0.642857,-0.0790569,-0.596559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,0.396059,0.148713,0.379565,0.23055,-0.749269,0.38305,-0.18307,0.0412656,0.244545,0.305047,...,0.225845,0.151985,0.521837,0.0374503,0.263123,1,0.250824,0.631651,0.323418,0.224754
262,0.207853,-0.0953772,0.250307,0.215754,0.866025,0.0338861,-0.346457,0.294877,0.119369,0.265621,...,0.271115,0.0262375,-0.0658351,0.209681,-0.10568,0.250824,1,0.320415,0.279291,0.731329
174,-0.0448963,0.0801638,0.563688,0.149548,-0.642857,-0.223839,0.0415603,0.170903,-0.165173,0.0215365,...,0.298998,-0.233787,0.118428,-0.17874,-0.110602,0.631651,0.320415,1,0.29218,0.495862
43,0.065407,-0.128217,0.0767775,0.229082,-0.0790569,0.152058,-0.0937368,0.368536,-0.000941202,0.045716,...,0.102915,0.0609546,-0.0424344,-0.102554,-0.221959,0.323418,0.279291,0.29218,1,0.316732


In [13]:
ratemat_short

,242,302,377,51,346,474,265,465,451,86,...,1594,1626,1645,1659,1682,1674,1640,1637,1630,1641
196,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,1,NaN,NaN,NaN,3,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,2,NaN,NaN,4,NaN,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,NaN,NaN,NaN,NaN,NaN,3,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,NaN,4,NaN,1,NaN,NaN,NaN,NaN,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
colab_filt.predict(user=196, item=242)
# 評価値の予測．

3.484374430184913

In [19]:
colab_filt.recommend(user=22, top_n=10, item_mst=movie_mst, key="movie_id")
# 指定したユーザーに上位10個の映画をレコメンド

,movie_id,predicted_score,movie_name
0,100,3.888195,Fargo (1996)
1,12,3.828325,"Usual Suspects, The (1995)"
2,56,3.823021,Pulp Fiction (1994)
3,98,3.695657,"Silence of the Lambs, The (1991)"
4,347,3.682624,Wag the Dog (1997)
5,475,3.619013,Trainspotting (1996)
6,654,3.619013,Chinatown (1974)
7,185,3.612553,Psycho (1960)
8,7,3.560246,Twelve Monkeys (1995)
9,300,3.552233,Air Force One (1997)
